In [1]:
import pandas as pd
import json
import os
import numpy as np

with open(r'C:\Users\acer\Desktop\Ds\Data 2018-2023\Project\2023\202300222.json') as fd:
    sc = json.load(fd)


In [2]:
# # Set option to display all rows
# pd.set_option('display.max_rows', None)

# # Set option to display full content in each cell without truncation
# pd.set_option('display.max_colwidth', None)

# # Optionally, set the display width to accommodate wide columns
# pd.set_option('display.width', None)

In [3]:
sc

{'abstracts-retrieval-response': {'item': {'ait:process-info': {'ait:status': {'@state': 'update',
     '@type': 'core',
     '@stage': 'S300'},
    'ait:date-delivered': {'@day': '25',
     '@timestamp': '2023-07-25T13:46:01.000001-04:00',
     '@year': '2023',
     '@month': '07'},
    'ait:date-sort': {'@day': '15', '@year': '2023', '@month': '10'}},
   'xocs:meta': {'xocs:funding-list': {'@pui-match': 'primary',
     '@has-funding-info': '1',
     'xocs:funding': {'xocs:funding-agency-matched-string': 'Chulalongkorn Academic Advancement'},
     'xocs:funding-addon-generated-timestamp': '2023-07-19T04:28:48.577Z',
     'xocs:funding-text': 'The Program Management Unit for Human Resources & Institutional Development, Research, and Innovation ( B16F640166 ). R.K. thanks Chulalongkorn Academic Advancement into its 2nd Century Project for Postdoctoral Fellowship.',
     'xocs:funding-addon-type': 'http://vtw.elsevier.com/data/voc/AddOnTypes/50.7/aggregated-refined'}},
   'bibrecord': {'

In [4]:
au_list = []

# Navigate to 'author-group' and ensure it's a list
author_groups = sc.get('abstracts-retrieval-response', {}) \
                  .get('item', {}) \
                  .get('bibrecord', {}) \
                  .get('head', {}) \
                  .get('author-group', [])

if isinstance(author_groups, dict):
    author_groups = [author_groups]

for group in author_groups:
    authors = group.get('author', [])
    if not authors:
        continue  # Skip if there are no authors
    
    affiliation = group.get('affiliation', {})
    if not affiliation:
        continue  # Skip if affiliation is missing
    
    aff_id = affiliation.get('@afid', '')
    
    # Extract organization names
    org = affiliation.get('organization', [])
    if isinstance(org, dict):
        aff_name = org.get('$', '')
    elif isinstance(org, list):
        aff_name = ', '.join(item.get('$', '') for item in org)
    else:
        aff_name = ''
    
    aff_city = affiliation.get('affiliation-city', affiliation.get('city', ''))
    aff_country = affiliation.get('affiliation-country', affiliation.get('country', ''))
    
    for author in authors:
        indexed_name = author.get('ce:indexed-name', 'Unknown')
        au_list.append((indexed_name, aff_id, aff_name, aff_city, aff_country))

In [5]:
au_list

[('Khezri R.',
  '60028190',
  'Department of Chemical Engineering, Faculty of Engineering, Chulalongkorn University',
  'Bangkok',
  'Thailand'),
 ('Motlagh S.R.',
  '60028190',
  'Department of Chemical Engineering, Faculty of Engineering, Chulalongkorn University',
  'Bangkok',
  'Thailand'),
 ('Etesami M.',
  '60028190',
  'Department of Chemical Engineering, Faculty of Engineering, Chulalongkorn University',
  'Bangkok',
  'Thailand'),
 ('Kheawhom S.',
  '60028190',
  'Department of Chemical Engineering, Faculty of Engineering, Chulalongkorn University',
  'Bangkok',
  'Thailand'),
 ('Mohamad A.A.',
  '60090694',
  'School of Materials and Mineral Resources Engineering, Universiti Sains Malaysia',
  'Nibong Tebal',
  'Malaysia'),
 ('Pornprasertsuk R.',
  '60028190',
  'Department of Materials Science, Faculty of Science, Chulalongkorn University',
  'Bangkok',
  'Thailand'),
 ('Pornprasertsuk R.',
  '60199582',
  'Center of Excellence in Petrochemical and Materials Technology, Chu

In [6]:
df_authors = pd.DataFrame(au_list, columns=['name', 'id', 'organization', 'city', 'country'])

In [7]:
df_authors

,name,id,organization,city,country
0,Khezri R.,60028190,"Department of Chemical Engineering, Faculty of...",Bangkok,Thailand
1,Motlagh S.R.,60028190,"Department of Chemical Engineering, Faculty of...",Bangkok,Thailand
2,Etesami M.,60028190,"Department of Chemical Engineering, Faculty of...",Bangkok,Thailand
3,Kheawhom S.,60028190,"Department of Chemical Engineering, Faculty of...",Bangkok,Thailand
4,Mohamad A.A.,60090694,School of Materials and Mineral Resources Engi...,Nibong Tebal,Malaysia
5,Pornprasertsuk R.,60028190,"Department of Materials Science, Faculty of Sc...",Bangkok,Thailand
6,Pornprasertsuk R.,60199582,Center of Excellence in Petrochemical and Mate...,Bangkok,Thailand
7,Pornprasertsuk R.,60018352,Department of Materials Science and Bioenginee...,Niigata,Japan
8,Pornprasertsuk R.,60028190,Center of Excellence on Advanced Materials for...,Bangkok,Thailand
9,Kheawhom S.,60028190,Center of Excellence on Advanced Materials for...,Bangkok,Thailand


In [8]:
references = sc['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference']

ref_list = []
for ref in references:
    ref_id = ref.get('@id', '')
    fulltext = ref.get('ref-fulltext', '')
    publication_year = ref.get('ref-info', {}).get('ref-publicationyear', {}).get('@first', '')
    title = ref.get('ref-info', {}).get('ref-title', {}).get('ref-titletext', '')
    source_title = ref.get('ref-info', {}).get('ref-sourcetitle', '')
    
    authors = ref.get('ref-info', {}).get('ref-authors', {}).get('author', [])
    author_names = [author.get('ce:indexed-name', '') for author in authors]
    authors_str = ', '.join(author_names)
    
    ref_list.append({
        'id': ref_id,
        'fulltext': fulltext,
        'year': publication_year,
        'title': title,
        'source_title': source_title,
        'authors': authors_str
    })

In [9]:
ref_list

[{'id': '1',
  'fulltext': 'Borchers, N., Clark, S., Horstmann, B., Jayasayee, K., Juel, M., Stevens, P., Innovative zinc-based batteries. J Power Sources, 484, 2021, 229309, 10.1016/j.jpowsour.2020.229309.',
  'year': '2021',
  'title': 'Innovative zinc-based batteries',
  'source_title': 'J Power Sources',
  'authors': 'Borchers N., Clark S., Horstmann B., Jayasayee K., Juel M., Stevens P.'},
 {'id': '2',
  'fulltext': 'Wang, N., Wan, H., Duan, J., Wang, X., Tao, L., Zhang, J., et al. A review of zinc-based battery from alkaline to acid. Mater Today Adv, 11, 2021, 100149, 10.1016/j.mtadv.2021.100149.',
  'year': '2021',
  'title': 'A review of zinc-based battery from alkaline to acid',
  'source_title': 'Mater Today Adv',
  'authors': 'Wang N., Wan H., Duan J., Wang X., Tao L., Zhang J.'},
 {'id': '3',
  'fulltext': 'Li, H., Ma, L., Han, C., Wang, Z., Liu, Z., Tang, Z., et al. Advanced rechargeable zinc-based batteries: recent progress and future perspectives. Nano Energy 62 (2019), 

In [10]:
ref_df = pd.DataFrame(ref_list)

In [11]:
ref_df

,id,fulltext,year,title,source_title,authors
0,1,"Borchers, N., Clark, S., Horstmann, B., Jayasa...",2021,Innovative zinc-based batteries,J Power Sources,"Borchers N., Clark S., Horstmann B., Jayasayee..."
1,2,"Wang, N., Wan, H., Duan, J., Wang, X., Tao, L....",2021,A review of zinc-based battery from alkaline t...,Mater Today Adv,"Wang N., Wan H., Duan J., Wang X., Tao L., Zha..."
2,3,"Li, H., Ma, L., Han, C., Wang, Z., Liu, Z., Ta...",2019,Advanced rechargeable zinc-based batteries: re...,Nano Energy,"Li H., Ma L., Han C., Wang Z., Liu Z., Tang Z."
3,4,"Etesami, M., Khezri, R., Abbasi, A., Nguyen, M...",2022,Ball mill-assisted synthesis of NiFeCo-NC as b...,J Alloys Compd,"Etesami M., Khezri R., Abbasi A., Nguyen M.T.,..."
4,5,"Wang, J., Yang, Y., Zhang, Y., Li, Y., Sun, R....",2021,Strategies towards the challenges of zinc meta...,Energy Storage Mater,"Wang J., Yang Y., Zhang Y., Li Y., Sun R., Wan..."
5,6,"Tie, Z., Niu, Z., Design strategies for high-p...",2020,Design strategies for high-performance aqueous...,Angew Chem Int Ed,"Tie Z., Niu Z."
6,7,"Radenahmad, N., Khezri, R., Mohamad, A.A., Ngu...",2021,A durable rechargeable zinc-air battery via se...,J Alloys Compd,"Radenahmad N., Khezri R., Mohamad A.A., Nguyen..."
7,8,"Duan, A., Luo, S., Sun, W., Insight into the d...",2023,Insight into the development of electrolytes f...,Chin Chem Lett,"Duan A., Luo S., Sun W."
8,9,"Yu, Y., Xu, W., Liu, X., Lu, X., Challenges an...",2020,Challenges and strategies for constructing hig...,Adv Sustain Syst,"Yu Y., Xu W., Liu X., Lu X."
9,10,"Yang, Q., Li, Q., Liu, Z., Wang, D., Guo, Y., ...",2020,Dendrites in Zn-based batteries,Adv Mater,"Yang Q., Li Q., Liu Z., Wang D., Guo Y., Li X."


In [12]:
import os
import json
import pandas as pd

folder_path = r'C:\Users\acer\Desktop\Ds\Data 2018-2023\Project\2023'

data = []

for file_name in os.listdir(folder_path):
    if file_name.endswith('.json'):
        with open(os.path.join(folder_path, file_name), 'r', encoding='utf-8') as fd:
            sc = json.load(fd)
            subject_areas = sc['abstracts-retrieval-response']['subject-areas']['subject-area']
            if isinstance(subject_areas, list):
                for subject in subject_areas:
                    data.append({
                        'file_name': file_name,
                        'fa': subject.get('@_fa', ''),
                        'subject': subject.get('$', ''),
                        'code': subject.get('@code', ''),
                        'abbrev': subject.get('@abbrev', '')
                    })
            else:
                subject = subject_areas
                data.append({
                    'file_name': file_name,
                    'fa': subject.get('@_fa', ''),
                    'subject': subject.get('$', ''),
                    'code': subject.get('@code', ''),
                    'abbrev': subject.get('@abbrev', '')
                })

subject_df = pd.DataFrame(data)

In [13]:
subject_df

,file_name,fa,subject,code,abbrev
0,202300000.json,true,Structural Biology,1315,BIOC
1,202300000.json,true,Biochemistry,1303,BIOC
2,202300000.json,true,Molecular Biology,1312,BIOC
3,202300001.json,true,Analytical Chemistry,1602,CHEM
4,202300001.json,true,Spectroscopy,1607,CHEM
...,...,...,...,...,...
7624,202302888.json,true,Ecology,2303,ENVI
7625,202302888.json,true,Aquatic Science,1104,AGRI
7626,202302889.json,true,"Geography, Planning and Development",3305,SOCI
7627,202302889.json,true,Development,3303,SOCI


In [14]:
def check_subjects(df, subjects):
    """
    Checks if the specified subjects exist in the DataFrame and identifies the files they are in.

    Parameters:
    - df (pd.DataFrame): DataFrame with 'file_name' and 'subject_area' columns.
    - subjects (list): List of subject names to search for.

    Returns:
    - matched_files (list): List of unique file names containing the specified subjects.
    - df_matched (pd.DataFrame): DataFrame listing file names and their matched subjects.
    """
    matched_files = []
    matched_data = []

    for subject in subjects:
        # Perform case-insensitive matching
        matches = df[df['subject'].str.lower() == subject.lower()]
        
        if not matches.empty:
            # Get unique file names where the subject exists
            files = matches['file_name'].unique().tolist()
            matched_files.extend(files)
            
            # Append matched subjects and corresponding file names
            for file in files:
                matched_data.append({'file_name': file, 'subject': subject})
        else:
            print(f"Subject '{subject}' does not exist.")

    # Remove duplicate file names
    matched_files = list(set(matched_files))

    # Create a DataFrame for matched subjects and files
    df_matched = pd.DataFrame(matched_data)

    return matched_files, df_matched

In [15]:
subjects_to_search = ['finance','Economics and Econometrics','Economics, Econometrics and Finance (all)']
matched_files, df_matched_subjects = check_subjects(subject_df, subjects_to_search)

In [16]:
print("Files containing the specified subjects:")
print(matched_files)

Files containing the specified subjects:
['202301404.json', '202302567.json', '202300754.json', '202301608.json', '202302008.json', '202300388.json', '202302637.json', '202301607.json', '202300262.json', '202302245.json', '202301518.json', '202301503.json', '202301036.json', '202301568.json', '202302273.json', '202302338.json', '202302639.json', '202302009.json', '202300352.json', '202300277.json', '202301221.json', '202301544.json', '202301815.json', '202302616.json', '202300719.json', '202301193.json', '202302337.json', '202302836.json', '202302091.json', '202300773.json', '202302416.json', '202301972.json', '202302883.json', '202300827.json', '202300617.json', '202301005.json', '202300999.json', '202301557.json', '202302003.json', '202301174.json', '202300063.json', '202300371.json', '202302259.json', '202302570.json', '202301514.json', '202302870.json', '202301441.json', '202301037.json', '202301012.json', '202300334.json', '202302638.json', '202301923.json', '202302848.json', '202

In [17]:
df_matched_subjects

,file_name,subject
0,202300277.json,finance
1,202300334.json,finance
2,202300719.json,finance
3,202300754.json,finance
4,202300827.json,finance
...,...,...
67,202302638.json,"Economics, Econometrics and Finance (all)"
68,202302639.json,"Economics, Econometrics and Finance (all)"
69,202302721.json,"Economics, Econometrics and Finance (all)"
70,202302836.json,"Economics, Econometrics and Finance (all)"


In [18]:
ref_list = []

for file_name in matched_files:
    # Ensure full path to the file
    full_path = os.path.join(folder_path, file_name)
    
    # Check if the file exists before opening it
    if os.path.exists(full_path):
        with open(full_path, 'r', encoding='utf-8') as file:
            sc = json.load(file)

        references = sc['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference']

        for ref in references:
            ref_id = ref.get('@id', '')
            fulltext = ref.get('ref-fulltext', '')
            publication_year = ref.get('ref-info', {}).get('ref-publicationyear', {}).get('@first', '')
            title = ref.get('ref-info', {}).get('ref-title', {}).get('ref-titletext', '')
            source_title = ref.get('ref-info', {}).get('ref-sourcetitle', '')

            authors = ref.get('ref-info', {}).get('ref-authors', {}).get('author', [])
            author_names = [author.get('ce:indexed-name', '') for author in authors]
            authors_str = ', '.join(author_names)

            ref_list.append({
                'id': ref_id,
                'fulltext': fulltext,
                'year': publication_year,
                'title': title,
                'source_title': source_title,
                'authors': authors_str
            })
    else:
        print(f"File not found: {full_path}")


In [19]:
ref_list

[{'id': '1',
  'fulltext': 'Aaker, D. A., & Mascarenhas, B. (1984). The need for strategic flexibility. The Journal of Business Strategy, 5(2), 74.',
  'year': '1984',
  'title': 'The need for strategic flexibility',
  'source_title': 'The Journal of Business Strategy',
  'authors': 'Aaker D.A., Mascarenhas B.'},
 {'id': '2',
  'fulltext': 'Abbott, A., & Banerji, K. (2003). Strategic flexibility and firm performance: the case of US based transnational corporations. Global Journal of Flexible Systems Management, 4(1-2), 1-8.',
  'year': '2003',
  'title': 'Strategic flexibility and firm performance: The case of US based transnational corporations',
  'source_title': 'Global Journal of Flexible Systems Management',
  'authors': 'Abbott A., Banerji K.'},
 {'id': '3',
  'fulltext': 'Bahrami, H., & Evans, S. (2011). Super-flexibility for real-time adaptation: Perspectives from Silicon Valley. California Management Review, 53(3), 21-39.',
  'year': '2011',
  'title': 'Super-flexibility for r

In [20]:
ref_df = pd.DataFrame(ref_list)

# Replace empty strings with NaN
ref_df.replace('', np.nan, inplace=True)

ref_df.dropna(inplace=True)

# Optionally, reset the index after dropping rows
ref_df.reset_index(drop=True, inplace=True)

In [21]:
ref_df

,id,fulltext,year,title,source_title,authors
0,1,"Aaker, D. A., & Mascarenhas, B. (1984). The ne...",1984,The need for strategic flexibility,The Journal of Business Strategy,"Aaker D.A., Mascarenhas B."
1,2,"Abbott, A., & Banerji, K. (2003). Strategic fl...",2003,Strategic flexibility and firm performance: Th...,Global Journal of Flexible Systems Management,"Abbott A., Banerji K."
2,3,"Bahrami, H., & Evans, S. (2011). Super-flexibi...",2011,Super-flexibility for real-time adaptation: Pe...,California Management Review,"Bahrami H., Evans S."
3,4,"Barney, J. (1991). Firm resources and sustaine...",1991,Firm resources and sustained competitive advan...,Journal of Management,Barney J.
4,5,"Bhagat, S., & Black, B. (2001). The non-correl...",2001,The non-correlation between board independence...,Journal of Corporate Law,"Bhagat S., Black B."
...,...,...,...,...,...,...
2305,28,"Suttiwongpan, C., Tochaiwat, K. & Naksuksakul,...",2019,Influence of Designs Following Green Assessmen...,ABAC Journal,"Suttiwongpan C., Tochaiwat K., Naksuksakul S."
2306,30,"Thaker, H. M. T., & Sakaran, K. C. (2016). Pri...",2016,Prioritization of key attributes influencing t...,International Journal of Housing Markets and A...,"Thaker H. M. T., Sakaran K. C."
2307,34,"Tochaiwat, K., Hankamolsiri, N., & Larpcharoen...",2021,Development of balanced scorecard for real est...,International Transaction Journal of Engineeri...,"Tochaiwat K., Hankamolsiri N., Larpcharoen V."
2308,35,"Tochaiwat, K., Likitanupak, W., & Kongsuk, S. ...",2017,Location selection models for low-rise condomi...,"Veridian E-Journal, Silpakorn University","Tochaiwat K., Likitanupak W., Kongsuk S."
